In [1]:
import numpy as np
import math
from scipy.optimize import curve_fit

In [2]:
print("\n \n")
print("------------------------------------ GEM 1 : ENCODER AZIMUTH CORRECTION --------------------------------------------------------")
print("\n")


 

------------------------------------ GEM 1 : ENCODER AZIMUTH CORRECTION --------------------------------------------------------




In [4]:
def main():
	
	jlist = int(input("Single file (1) or Multiple files (2)?: \n"))
	print("\n")
	
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

	if jlist == 1:
		datetag = int(input("Enter the date of the year [yy_MM_dd] : \n"))
		jfilenum = int(input("Enter the UT time of the day [hhmmssml] : \n"))
		jj = 1
	else:
		filelist = input("ENTER the filelist name: \n")
		print("\n")
		jj = 0
		
	jset = int(input(" SET Vmax? [1] => yes / [2] => no \n"))
	print("\n")

	if jset == 1:
		Vmax = float(input(" Write Vmax: \n"))
	else:
		Vmax = 9.8993 # 1999 observations
		
	jjset = int(input("SET slp rate? [1] => yes / [2] => no \n"))
	print("\n")
	if jjset == 1:
		slp = float(input("Write slp rate: \n"))
	else:
		slp = 17.0357 # iterated 1999 Vmax +/- 0.3918
		
	numframes = int(input("ENTER the expected number of frames/scan sequence :[1994 > 106; 1999 > 105; 2005 > 103,104] \n"))
	
	Dir = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/txt/" # filelist dir   
	Dir1 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/dat/"
	Dir2 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/rots/"
	Dir3 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/azimuth/"
	Dir4 = "/media/nicolli/eabdalla2012/resultados_nicolli/tradução_gem_modulos/rots/dpf/"
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
	with open(Dir + filelist) as arq:
		filelist = arq.readlines()
		n_files = len(filelist)
		file_list, datetag, filenum, tag = [], [], [], []
		for i in range(n_files):
			s_datetag = str(filelist[i])
			datetag.append(s_datetag[:8])
			filenum.append(s_datetag[8:16])
			tag.append(s_datetag)
			file_list.append(filelist[i])
		N_files = len(filelist)	
#::::::::::::::::::::: Start Azimuth correction :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::	
	jframe, time, azimut, firstime, compaz, ttime, aazimut, sig, jccor = [], [], [], [], [], [], [], [], []
	j, k, l, ic, j105, j106, timemean, aazmean = 0, 0, 0, 0, 0, 0, 0, 0
	firstazi, lastazi, midazi = 0,0,0
	first, dangling = True, False 	
	for file_name_index in range(n_files): # file loop
		print("Doing file: \n", tag[file_name_index])
		file_name = filelist[file_name_index]
		file_name = file_name.replace("\t", "")
		file_name = file_name.replace("\n", "")

        with open(Dir5 + file_name, "r+") as f:
			input_data = f.readlines()
			n_lines = len(input_data)
            for line in input_data:
                input_line = line.replace(" ", "")
				input_line = line.replace("\n", "")
				input_line = input_line.split(" ")
				input_line = list(filter(None, input_line))
				jframe.append(input_line[0])
				time.append(input_line[1])
                azimut.append(input_line[4])
                
		i_frames = len(jframe)
		for i in range(i_frames):
			slope,dslope,aazi,dpfazi = [],[],[],[]
			if first: # Get a reference in time and direction.
				compaz = azimut[0]   
				firstime = time
				first = False
				
		#this is the expected behavior, except for the very 
		#last azimut which may differ from the previous one
		#by < 176 and yet show reversal in sense of 
		#direction. The usual is a continuous decrease in 
		#azimut. An unexpected sign reversal can also happen. =>
			if azimut[i]-compaz <= 176: 
				
				if i == 0:
					l = 0
				else:
					l += 1
				
				jl = l - 1
				if (l >= 1) and (time[i] - time[jl] <= 0):
					ttime.append(time[i] + 86400)
				else:
					ttime.append(time[i])
				l_ant = l-1
				aazimut = azimut
				timemean = timemean + ttime[l]
				aazmean = aazmean + azimut[i]
				sig.append(1)
				
				if azimut[i]-compaz == 0:
					if l >= (numframes-1):
						dangling = True #sign reversal in last azimut detected 
					else:
						jccor.append(l)                      
			else:
				if l >= (numframes-1):
					if l == (numframes-1):
						j105 += 1
					if l == numframes:
						j106 += 1
					j += 1
					
					if dangling:
						mm = 1
						slope,dslope,aazi = dpf(timemean,aazmean,ttime,aazimut,sig,slope,dslope,azimut,l,mm)
						lastazi = a
						midazi = aazimut[l-1]
						firstazi = aazimut[l]
						l2 = 1
					else:
						lastazi = aazimut[l]
						mm = 0
						slope,dslope,aazi = dpf(timemean,aazmean,ttime,aazimut,sig,slope,dslope,azimut,l,mm)
						midazi = aazimut[l]
						jj = l+1
						firstazi = aazimut[jj]
						l2 = 2
					speed, dspeed, dpfazi = [],[],[]
					speed.append(slope)
					dspeed.append(dslope) 
					#If the Vmax used is correct, then midazi will be
				        #spaced evenly from lastazi to firstazi according
				        #to the prevailing slope. So we check and correct
					dpfazi.append((360.0 + lastazi - firstazi)/2)
					print(speed,dspeed,dpfazi)
		 			
					if ic > 0: # ic value = times unexpected signal reverse occurs
						for i in range(ic):
							ind_azi_corr = jccor[i]
							ind_ant_azi = jccor[i] - 1
							aazimut[ind_azi_corr] = aazimut[ind_ant_az] + slope*0.56002
							if aazimut[ind_azi_corr] < 0: aazimut[ind_azi_corr] += 360.0
					
					data = str(j)
					for item in [slope, dslope, lastazi, midazi, firstazi, dpfazi[j]]:
						item = f"{item:.2f}"
						data += f"\t{item}\t"
					file_rots = file_list.replace(".dat", "rots.dat")
					
					with open(Dir2 + file_rots, "w") as arq1: # write rotation parameters
							arq1.write(data)
							arq1.write("\n") 
						
				else:
					print("Incomplete rotation of l frames =>",l)
					if ic > 0:
						mm = 1
						slope,dslope,aazi = dpf(timemean,aazmean,ttime,aazimut,sig,slope,dslope,azimut,l,mm)					
						l2 = 1
					else:
						mm = 0
						slope,dslope,aazi = dpf(timemean,aazmean,ttime,aazimut,sig,slope,dslope,azimut,l,mm)
						l2 = 2
					#speed, dspeed, dpfazi = [],[],[]
					#speed.append(slope)
					#dspeed.append(dslope) 
					#dpfazi.append(360.0 + lastazi - firstazi)/2 
				fpd =   slope*0.56002 # frames per degree
				ww = l+l2
				d = 1
				print("slopw",slope)
				print("fpd",fpd)
				print("aazimut")
				
				# trocar aazimut por aazi daqui pra baixo =>
				# mas porque os dados da list aazimut estão c/ tempo? (talvez ordem errada na leitura)
				
				while d <= ww:
					if d > 1:
					
						if ((aazimut[d] - aazimut[d-1]) < (fpd/2)) or ((aazimut[d] - aazimut[d-1]) > (3*fpd/2))\
						and ((ttime[d] - time[d-1]) < (3*0,56002/2)):
							aazimut[d] = aazimut[d-1] + fpd
					d += 1
				print("azi data corrigido", aazimut)
				for item in aazimut:
					if aazimut[0]:
						dta_azi = str(aazimut[0])
						dta_azi += "\n"
					else:
						dta_azi += f"item\n"
						
				name_azi = file_name.replace(".dat", "azi.dat")
				with open(Dir3 + name_azi,"w") as arq2:
					arq2.write(dta_azi)
				
		while i <= l: #last incomplete rotation 
			if i > 1:
				if ((aazimut[i] - aazimut[i-1]) < (fpd/2)) or ((aazimut[i] - aazimut[i-1]) > (3*fpd/2))\
				and ((ttime[i] - time[i-1]) < (3*0,56002/2)):
					aazimut[i] = aazimut[i-1] + fpd
			i += 1
					
		for item in aazimut:
			if aazimut[0]:
				dta_azi = str(aazimut[0])
				dta_azi += "\n"
			else:
				dta_azi += f"item\n"
						
		name_azi = file_name.replace(".dat", "azi.dat")
		with open(Dir3 + name_azi,"w") as arq3:
			arq3.write(dta_azi)
#:::::::::::::::::::::::::::::::::::::::::End azimuth correction:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# Statistics rotation
		#dpfazi = 0
		slopmean,sigma,sigint,Z,dZ,dpfmean,sigdpf,fpd,sig_dpf,sigman_dpf,sigint_dpf,\
		mean_per,per_sigman,sigma_per,sigint_per,Vslp = stats_parameters(dpfazi,speed,dspeed,j)
		file_dpf = file_name.replace(".dat", "dpf_rots.dat")
		data_dpf = str(j)
		for item in [fpd,sig_dpf,dpfmean,sigdpf,(fpd-dpfmean),Vslp]:
			item = f" {item:.3f} "
			data_dpf += f"\t{item}"
		with open(Dir4 + file_dpf) as arq4:
			arq4.write(data_dpf)
			arq4.write("\n")
							
#------------------------------------------ End GEM 1 -----------------------------------------------------------------------------------

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 65)

In [ ]:
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::				
def stats_parameters(dpfazi,speed,dspeed,j):
	"""
	Calculates rotation statistics
	"""
	wt, slopemean, dpfmean, sigma, sigdpf = 0,0,0,0,0
	for i in range(j):
		w = 1.0/dspeed[i]**2
		slopemean += speed[i]*w
		wt += w 
		dpfmean += dpfazi[i]
	print("w",w)
	slopemean = slopemean/wt #mean dish rotation speed in deg/sec
	dpfmean = dpfmean/j ##Average dpf speed in deg/frame
	for i in range(j):
		w = 1.0/dspeed[i]**2
		sigma = sigma + w*(speed[i] - slopemean)**2    
		sigdpf = sigdpf + (dpfazi[i] - dpfmean)**2
			
	sigma = math.sqrt(sigma/wt*j/(j-1)) # sigma dispersion (slopemean) deg/sec
	sigman = sigma/math.sqrt(j*1) # external error (slopemean) deg/sec
	sigint = math.sqrt(1/wt) # internal error (slopemean) deg/sec            
	Z = sigman/sigint # z statistic
	dZ = 1/math.sqrt(2*(j-1))         
	sigdpf = math.sqrt(sigdpf/(j-1))/math.sqrt(j*1)
	
	fpd = slopemean*0.56002 # frames/degree
	sig_dpf = sigma * 0.56002 # sigma dispersion
	sigman_dpf = sigman * 0.56002 # external error
	sigint_dpf = sigint * 0.56002 # internal error
	
	mean_per = 360/slopemean # Average rotation period in seconds
	per_sigman = 360*sigman/slopemean**2 # external error
	sigma_per = 360*sigma/slopemean**2 # sigma dispersion
	sigint_per = 360*sigint/slopemean**2 # internal error
	
	Vslp = Vmax + (fpd - dpfmean)/slp
	return(slopmean,sigma,sigint,Z,dZ,dpfmean,sigdpf,fpd,sig_dpf,sigman_dpf,sigint_dpf,mean_per,per_sigman,sigma_per,sigint_per,Vslp)

In [ ]:
#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def dpf(tmean,zmean,ttim,aazi,sigg,slop,dslop,azi,n,m):
	"""
	Using the output of the least_sq function and the average values ​​
	of time and azimuth, calculate the speed in degrees/frame
	"""
	n_azi = len(azi)
	for fr_azi in range(n_azi):
		if m == 0: #se não há inversão de sinal
			tmean = tmean/(n+1)
			zmean = zmean/(n+1)
			jh = (n)+2-m
			aazi[jh] = azi[fr_azi]
		else: #se há
			tmean = (tmean - ttim[n])/((n+1)-1) 
			zmean = (zmean - aazi[n])/((n+1)-1)
			jh = (n+1)+2-m
			aazi[jh] = azi[fr_azi]	
		i = 1
		while i < (n+1)-m:
			ttim[i] = (ttim[i] - tmean)
			aazi[i] = (aazi[i] - zmean)
			i += 1
		ttim = np.array(ttim, dtype="f8")
		aazi = np.array(aazi, dtype="f8")
		if ((n+1)-m) > 2:
			mnt = 0
			zer, slope, dzer, dslope  = least_sq(function, ttim, aazi)	
		else:
			slope, dslope = 0, 0
					
		i = 1
		while i < (n+1)-m:
			aazi[i] += zmean
		aux = azi[fr_azi]
		jk = (n)-m
		if (aazi[jh]<=azi[fr_azi]) or (aazi[jh]-azi[fr_azi])<(slop*0.56002/8*7):
			dstep = (360+aazi[jk]- azi[fr_azi])/3
			for i in [0,1,2]:
				jr = (n)+i-m
				aazi[jr] = aazi[jk] - i*dstep
				if aazi[jr]<0:
					aazi[jr] = aazi[jr]
		else:
			jr = (n+1)+1-m
			ji = (n+1)+2-m
			aazi[jr] = aazi[jk] - (360 + aazi[jk] - aazi[ji])/2
			if aazi[jr]<0:
				aazi[jr] = aazi[jr]
			jg = (n+1)+3-m
			aazi[jg] = azi[fr_azi]
			n += 1
			
	return slope,dslope,aazi

In [ ]:
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::	
def least_sq(function,x,y):
	"""
	Calculate a linear fit to the azimuth data as a function of time,
	obtaining the speed in degrees/sec and their respective uncertainties
	"""
	param, m_cov = curve_fit(function, x, y)
	print("a,b", parameters)
	a = param[0], b = param[1]
	perr = np.sqrt(np.diag(pcov)) #siga, sigb
	print("siga, sigb", perr)
	siga, sigb = perr[0], perr[1]
	
	return a,b,siga,sigb

#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [ ]:
def function(x,a,b):
	return a + bx

#::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [ ]:
main()